In [1]:
import os
import pandas as pd
import numpy as np
import json
import re
import gensim
import nltk
import requests
import datetime
import seaborn as sns
import math
import plotly.express as px

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [2]:
UK_DF = pd.read_csv('data/UK_Watch_Time.csv')
subsrevDF = pd.read_excel('data/netflix-subs-rev.xlsx')

In [3]:
Duration_DF = UK_DF.groupby(['title','genres']).sum()
Duration_DF.reset_index(inplace=True)

In [4]:
Duration_Ac_DF= Duration_DF.sort_values('duration', ascending = False)

In [5]:
Top_10 = Duration_Ac_DF.head(10)

In [6]:
Top_10.to_csv ('Top_10.csv', index = False)

In [7]:
Duration_De_DF= Duration_DF.sort_values('duration', ascending = True)

In [8]:
Duration_De_DF_NON = Duration_De_DF[(Duration_De_DF['duration']>0)]

In [9]:
Bot_10 = Duration_De_DF_NON.head(10)

In [10]:
#streaming_tvshows_df = pd.read_csv('streaming_tvshows.csv')
streaming_movies_df = pd.read_csv('data/streaming_movies.csv')

In [11]:
#netflix_movies_df = streaming_movie_df[(streaming_movie_df['Netflix']>0)]
#netflix_movies_year_df = netflix_movies_df.groupby(["Year"]).count()
netflix_movies_year_df = streaming_movies_df[["Year","Netflix"]].groupby(["Year","Netflix"], axis=0).size().reset_index(name='counts')
netflix_movies_year_df = netflix_movies_year_df[(netflix_movies_year_df['Netflix']>0)]

hulu_movies_year_df = streaming_movies_df[["Year","Hulu"]].groupby(["Year","Hulu"], axis=0).size().reset_index(name='counts')
hulu_movies_year_df = hulu_movies_year_df[(hulu_movies_year_df['Hulu']>0)]

prime_movies_year_df = streaming_movies_df[["Year","Prime Video"]].groupby(["Year","Prime Video"], axis=0).size().reset_index(name='counts')
prime_movies_year_df = prime_movies_year_df[(prime_movies_year_df['Prime Video']>0)]

disney_movies_year_df = streaming_movies_df[["Year","Disney+"]].groupby(["Year","Disney+"], axis=0).size().reset_index(name='counts')
disney_movies_year_df = disney_movies_year_df[(disney_movies_year_df['Disney+']>0)]
#netflix_movies_year_df.apply(lambda x: x['Netflix'] == 1)

In [12]:
netflix_movies_year_df['Site']='Netflix'
netflix_movies_year_df = netflix_movies_year_df.drop(columns='Netflix')

hulu_movies_year_df['Site']='Hulu'
hulu_movies_year_df = hulu_movies_year_df.drop(columns='Hulu')

prime_movies_year_df['Site']='Prime'
prime_movies_year_df = prime_movies_year_df.drop(columns='Prime Video')

disney_movies_year_df['Site']='Disney+'
disney_movies_year_df = disney_movies_year_df.drop(columns='Disney+')

In [13]:
frames=[netflix_movies_year_df, hulu_movies_year_df, prime_movies_year_df,disney_movies_year_df]

all_movies_year_df=pd.concat(frames)

# Score Cleaning

In [14]:
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac


In [15]:
streaming_movies_df['IMDb Cleaning'] = streaming_movies_df['IMDb'].fillna(0)
streaming_movies_df['IMDb Cleaning'] = streaming_movies_df['IMDb Cleaning'].apply(convert_to_float)
streaming_movies_df = streaming_movies_df[(streaming_movies_df['IMDb Cleaning'] > 0)]


streaming_movies_df['Rotten Tomatoes Cleaning'] = streaming_movies_df['Rotten Tomatoes'].fillna(0)
streaming_movies_df['Rotten Tomatoes Cleaning'] = streaming_movies_df['Rotten Tomatoes Cleaning'].apply(convert_to_float)
streaming_movies_df = streaming_movies_df[(streaming_movies_df['Rotten Tomatoes Cleaning'] > 0)]



In [16]:
Netflix_Score = streaming_movies_df[(streaming_movies_df['Netflix'] > 0)]
Hulu_Score = streaming_movies_df[(streaming_movies_df['Hulu'] > 0)]
Disney_Score = streaming_movies_df[(streaming_movies_df['Disney+'] > 0)]
Prime_Score = streaming_movies_df[(streaming_movies_df['Prime Video'] > 0)]

In [17]:
Netflix_Score["IMDb Cleaning"].mean()
Hulu_Score["IMDb Cleaning"].mean()
IMDb_Score = {'Streaming Sites': ["Netflix", "Hulu", "Disney", "Prime Video"], 
              'IMDb Score': [Netflix_Score["IMDb Cleaning"].mean(), Hulu_Score["IMDb Cleaning"].mean(),
                       Disney_Score["IMDb Cleaning"].mean(), Prime_Score["IMDb Cleaning"].mean()]}
IMDb_Score_df = pd.DataFrame(data=IMDb_Score)

In [18]:
RT_Score = {'Streaming Sites': ["Netflix", "Hulu", "Disney", "Prime Video"], 
              'Rotten Tomatoes Score': [Netflix_Score["Rotten Tomatoes Cleaning"].mean(), Hulu_Score["Rotten Tomatoes Cleaning"].mean(),
                       Disney_Score["Rotten Tomatoes Cleaning"].mean(), Prime_Score["Rotten Tomatoes Cleaning"].mean()]}
RT_Score_df = pd.DataFrame(data=RT_Score)

# Top/Bottom 10 in the UK

In [19]:
UK_DF['genres'] = UK_DF['genres'].astype('str')

In [20]:
processed_docs = UK_DF["genres"]

In [21]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize(token))
    return result
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')


In [22]:
processed_docs = processed_docs.map(preprocess)
processed_docs.head()

dictionary = gensim.corpora.Dictionary(processed_docs)

In [23]:
streaming_genre_df = streaming_movies_df["Genres"].astype(str)
streaming_genre_df = streaming_genre_df.map(preprocess)
streaming_dictionary_genre_df = gensim.corpora.Dictionary(streaming_genre_df)

# Getting Top Genre/Director/Actor (All Streaming Websites)

In [24]:
def cleaning (genre):
    return genre.replace(',', ' ')

def biography (genre):
    if genre.find("Biography") != -1:
        return 1
    else:
        return 0
def drama (genre):
    if genre.find("Drama") != -1:
        return 1
    else:
        return 0
def crime (genre):
    if genre.find("Crime") != -1:
        return 1
    else:
        return 0
def action (genre):
    if genre.find("Action") != -1:
        return 1
    else:
        return 0
def sport (genre):
    if genre.find("Sport") != -1:
        return 1
    else:
        return 0
def documentary (genre):
    if genre.find("Documentary") != -1:
        return 1
    else:
        return 0
def musical (genre):
    if genre.find("Musical") != -1:
        return 1
    else:
        return 0
def history (genre):
    if genre.find("History") != -1:
        return 1
    else:
        return 0
def romance (genre):
    if genre.find("Romance") != -1:
        return 1
    else:
        return 0
def comedy (genre):
    if genre.find("Comedy") != -1:
        return 1
    else:
        return 0
def adventure (genre):
    if genre.find("Adventure") != -1:
        return 1
    else:
        return 0
def mystery (genre):
    if genre.find("Mystery") != -1:
        return 1
    else:
        return 0
def western (genre):
    if genre.find("Western") != -1:
        return 1
    else:
        return 0
def thriller (genre):
    if genre.find("Thriller") != -1:
        return 1
    else:
        return 0
def music (genre):
    if genre.find("Music") != -1:
        return 1
    else:
        return 0
def animation (genre):
    if genre.find("Animation") != -1:
        return 1
    else:
        return 0
def fantasy (genre):
    if genre.find("Fantasy") != -1:
        return 1
    else:
        return 0
def horror (genre):
    if genre.find("Horror") != -1:
        return 1
    else:
        return 0
def family (genre):
    if genre.find("Family") != -1:
        return 1
    else:
        return 0
def short (genre):
    if genre.find("Short") != -1:
        return 1
    else:
        return 0
def family (genre):
    if genre.find("Thriller") != -1:
        return 1
    else:
        return 0
def talk (genre):
    if genre.find("Talk") != -1:
        return 1
    else:
        return 0
def reality (genre):
    if genre.find("Reality") != -1:
        return 1
    else:
        return 0
def news (genre):
    if genre.find("News") != -1:
        return 1
    else:
        return 0
def game (genre):
    if genre.find("Game") != -1:
        return 1
    else:
        return 0
def noir (genre):
    if genre.find("Noir") != -1:
        return 1
    else:
        return 0
def film (genre):
    if genre.find("Film") != -1:
        return 1
    else:
        return 0

In [25]:
streaming_movies_df["Genres Clean"] = streaming_movies_df["Genres"]

In [26]:
streaming_movies_df["Genres Clean"] = streaming_movies_df["Genres Clean"].astype(str)
streaming_movies_df["Genres Clean"] = streaming_movies_df["Genres Clean"].apply(cleaning)
streaming_movies_df["Biography"] = streaming_movies_df["Genres Clean"].apply(biography)
streaming_movies_df["Drama"] = streaming_movies_df["Genres Clean"].apply(drama)
streaming_movies_df["Crime"] = streaming_movies_df["Genres Clean"].apply(crime)
streaming_movies_df["Action"] = streaming_movies_df["Genres Clean"].apply(action)
streaming_movies_df["Sport"] = streaming_movies_df["Genres Clean"].apply(sport)
streaming_movies_df["Documentary"] = streaming_movies_df["Genres Clean"].apply(documentary)
streaming_movies_df["Musical"] = streaming_movies_df["Genres Clean"].apply(musical)
streaming_movies_df["History"] = streaming_movies_df["Genres Clean"].apply(history)
streaming_movies_df["Romance"] = streaming_movies_df["Genres Clean"].apply(romance)
streaming_movies_df["Comedy"] = streaming_movies_df["Genres Clean"].apply(comedy)
streaming_movies_df["Adventure"] = streaming_movies_df["Genres Clean"].apply(adventure)
streaming_movies_df["Mystery"] = streaming_movies_df["Genres Clean"].apply(mystery)
streaming_movies_df["Western"] = streaming_movies_df["Genres Clean"].apply(western)
streaming_movies_df["Thriller"] = streaming_movies_df["Genres Clean"].apply(thriller)
streaming_movies_df["Music"] = streaming_movies_df["Genres Clean"].apply(music)
streaming_movies_df["Animation"] = streaming_movies_df["Genres Clean"].apply(animation)
streaming_movies_df["Fantasy"] = streaming_movies_df["Genres Clean"].apply(fantasy)
streaming_movies_df["Horror"] = streaming_movies_df["Genres Clean"].apply(horror)
streaming_movies_df["Family"] = streaming_movies_df["Genres Clean"].apply(family)
streaming_movies_df["Short"] = streaming_movies_df["Genres Clean"].apply(short)
streaming_movies_df["Talk"] = streaming_movies_df["Genres Clean"].apply(talk)
streaming_movies_df["Reality"] = streaming_movies_df["Genres Clean"].apply(reality)
streaming_movies_df["News"] = streaming_movies_df["Genres Clean"].apply(news)
streaming_movies_df["Game"] = streaming_movies_df["Genres Clean"].apply(game)
streaming_movies_df["Film"] = streaming_movies_df["Genres Clean"].apply(film)
streaming_movies_df["Noir"] = streaming_movies_df["Genres Clean"].apply(noir)

In [27]:
streaming_movies_genrelist_df = streaming_movies_df

In [28]:
#streaming_movies_genrelist_df = streaming_movies_genrelist_df.drop("ID", axis = 1)
#streaming_movies_genrelist_df = streaming_movies_genrelist_df.drop("Year", axis = 1)
#streaming_movies_genrelist_df = streaming_movies_genrelist_df.drop("Age", axis = 1)
#streaming_movies_genrelist_df = streaming_movies_genrelist_df.drop("Unnamed: 0", axis = 1)
streaming_movies_genrelist_df = streaming_movies_genrelist_df.drop("Country", axis = 1)

In [29]:
total_genre = streaming_movies_genrelist_df
genre_array = {"Genre" : ["Biography", "Crime", "Drama"
                          , "Action", "Sport", "Documentary"
                          , "Musical", "History", "Romance", "Comedy"
                          , "Adventure", "Mystery", "Western", "Thriller"
                          , "Music", "Animation", "Fantasy", "Horror", "Family"
                          , "Short", "Talk", "Reality", "News", "Game"
                          , "Film", "Noir"],
               "Sum":[ total_genre["Biography"].sum(), total_genre["Crime"].sum(),
                        total_genre["Drama"].sum(), total_genre["Action"].sum(),
                        total_genre["Sport"].sum(), total_genre["Documentary"].sum(),
                        total_genre["Musical"].sum(), total_genre["History"].sum(),
                        total_genre["Romance"].sum(), total_genre["Comedy"].sum(),
                        total_genre["Adventure"].sum(), total_genre["Mystery"].sum(),
                        total_genre["Western"].sum(), total_genre["Thriller"].sum(),
                        total_genre["Music"].sum(), total_genre["Animation"].sum(),
                        total_genre["Fantasy"].sum(), total_genre["Horror"].sum(),
                        total_genre["Family"].sum(), total_genre["Short"].sum(),
                        total_genre["Talk"].sum(), total_genre["Reality"].sum(),
                        total_genre["News"].sum(), total_genre["Game"].sum(),
                        total_genre["Film"].sum(), total_genre["Noir"].sum()]}
genre_array_df = pd.DataFrame(data=genre_array)
genre_array_fig = px.bar(genre_array_df, x="Genre",y="Sum")

In [30]:
Drama_Movies = total_genre[(total_genre["Drama"]>0)]
Drama_Movies = Drama_Movies.sort_values('Rotten Tomatoes', ascending = False)
Genre_Top_Movies_df = Drama_Movies.head(10)

In [31]:
#data cleaning for netflix num of shows

netflixDF = pd.read_excel('data/netflix_titlesgenre-separated.xlsx')

# DATA CLEANING
netflixDF['date_added'] = pd.to_datetime(netflixDF['date_added'])
netflixDF['year_added'] = netflixDF['date_added'].dt.year.astype('Int64')
netflixDF['month_added'] = netflixDF['date_added'].dt.month.astype('Int64')
netflix_show = netflixDF.groupby(['type','year_added']).size().reset_index(name='count')


In [32]:
import plotly.express as px
Rottentomatoes2 = px.bar(RT_Score_df, x = "Streaming Sites", y = "Rotten Tomatoes Score")
IMDb_Graph2 = px.bar(IMDb_Score_df, x = "Streaming Sites", y = "IMDb Score")
#all_movies_df = px.data.gapminder().query("continent == 'Oceania'")
stream_release = px.line(all_movies_year_df,"Year","counts",color="Site")
#hululine = px.line(hulu_movies_year_df,"Year","counts")
#px.line(prime_movies_year_df,"Year","counts")
#px.line(disney_movies_year_df,"Year","counts")

In [ ]:
# -*- coding: utf-8 -*-

# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.
from jupyter_dash import JupyterDash
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table 
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


external_stylesheets = [#'https://codepen.io/chriddyp/pen/bWLwgP.css'
    {
        'href': 'https://cdn.jsdelivr.net/npm/bootstrap@5.1.0/dist/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-KyZXEAg3QhqLMpG8r+8fhAXLRk2vvoC2f3B09zVXn8CA5QIVfZOJ3BCsw2P0p/We',
        'crossorigin': 'anonymous',      
    }
]


#external_stylesheets=[dbc.themes.DARKLY]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#CSS CODES
#style for title
title_img = {
    "position":"center",
    "height":400,
    "width":850
}

#style for header
header_img = {
    "position":"fixed",
    #"position": "sticky",
    "top":0,
    "left":0,
    "bottom":0,
}

header2_img = {
    "position":"fixed",
    "top":0,
    "left":0,
    "bottom":0,
    "margin":0,
    "width":3005,
    "height":74
}

content_style = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

#style for a paragraph
p_style = {
    #'line-height':'1.0',
    #'font-family':'montserrat',
    'textAlign':'justify',
    'text-indent':'5%',
}

#style for tabs (on tab)
tabsOn_style = {
    #"borderTop":"white",
    "backgroundColor":"#049c9c",
    "color":"white",
    "borderBottom": "7px solid #d6d6d6",
    "borderTop":"7px solid #d6d6d6",
    "borderRight": "7px solid #d6d6d6",
    "borderLeft": "7px solid #d6d6d6"
}

#style for tabs (off tab)
tabsOff_style = {
    "borderBottom": "7px solid #d6d6d6",
    "borderTop":"7px solid #d6d6d6",
    "borderRight": "7px solid #d6d6d6",
    "borderLeft": "7px solid #d6d6d6",
    #"padding": "4px 4px 0px 0px"
}

#style for sidebar (body)
sidebar_style = {
    "position":"fixed",
    #"position": "sticky",
    "top":100,
    "left":0,
    "bottom":0,
    #"height": "10rem",
    "width": "16rem",
    #"padding": "2rem 1rem",
    "background-color": "#f8f9fa",
    "overflox":"scroll",
#     "text-align":"left", 
#     "border-bottom": "10px solid #555",
    "border-right": "5px solid #d6d6d6"
}

# #style for radio buttons
radio_style={
    'margin-right':10,
    'margin-left':10,
    'display': 'inline-block'
}

#style for sidebar (contents)
sidebarC_style = {
    "background-color":"#049c9c",
    "color": "white",
    "text-align":"left", 
    "border-bottom": "5px solid #d6d6d6",
    "padding":"10px 10px",
    #"border-right": "5px solid #d6d6d6"
    "font-size":"20px"
}

sidebar = html.Div([
        dbc.Nav(
            [
                dbc.NavLink("HOME",href="#start",active ="True", style=sidebarC_style),#Home
                dbc.NavLink("STREAMING SERVICES TREND", href="#Trend", external_link=True, style=sidebarC_style),#Streaming Services Trend
                dbc.NavLink("EFFECT OF THE PANDEMIC", href="#Pandemic", external_link=True, style=sidebarC_style),#Effect of the Pandemic
                dbc.NavLink("NETFLIX IN THE UK", href="#Net", external_link=True, style=sidebarC_style),#Netflix in the UK
            ],
            vertical = True,
            pills=True
        )
    ],style = sidebar_style
)

#START OF WEBPAGE CODE
content = html.Div([
    html.Img(src="...", style=title_img),
    #html.H1(children="How the Pandemic Affects the Streaming Services",id="start", style={'textAlign':'left','color':colors['text']}),
    html.Br(),
    html.Br(),
    html.P(id='Trend',children="""With the pandemic going on, it changes the lives of many people and how the 
    different industries work. It includes movie theaters having no choice but to halt operations. What rose from
    the lack of movie theaters is the number of streaming services that are available now. Given the current
    situation, streaming services has been on the rise and this group wanted to see how the pandemic has 
    affected said streaming services in vairous ways.""", style=p_style),
    html.Br(),
    html.Br(),
    
#STREAMING SERVICES TREND
    html.H3(children="STREAMING SERVICES TREND", style={'textAlign':'left','color':colors['text']}),
    html.Br(),
    html.P(children="To enable us to understand how the pandemic affects the streaming services, it would be best to check the different trends of these streaming services.", style=p_style),
    html.Br(),
    html.Div(className="Streaming-ServicesCN", children=[
        dcc.Tabs(id="streaming-tabs", value='tab-1', children=[
        dcc.Tab(label='NUMBER OF SHOWS RELEASE', value='tab-1', style=tabsOff_style, selected_style=tabsOn_style),
        dcc.Tab(label='AVERAGE SCORE OF EACH STERAMING SERVICE GET', value='tab-2', style=tabsOff_style, selected_style=tabsOn_style),
        dcc.Tab(label='TOP MOVIES PER GENRE', value='tab-3', style=tabsOff_style, selected_style=tabsOn_style),
        ])
    ]),
    html.Div(id='streaming-servicesID'),
    html.Br(id='Pandemic'),
    
    html.Br(),
    html.Br(),

#EFFECTS OF THE PANDEMIC
    html.H3(children="EFFECT OF THE PANDEMIC", style={'textAlign':'left','color':colors['text']}),
    html.Br(),
    html.P(children="""Now, we will take a look into the the statistics of Netflix and how the pandemic has 
    affected its number of shows added, quarterly revenue from 2018-2021, and the number of subsribers
    Netflix has garnered quarterly from 2018-2021.""", style=p_style),
    html.Br(),
    html.Div(className="Effects-PandemicCN", children=[
        dcc.Tabs(id="effects-tabs", value='tab-1', children=[
        dcc.Tab(label='NUMBER OF SHOWS ADDED IN NETFLIX PER YEAR', value='tab-1', style=tabsOff_style, selected_style=tabsOn_style),
        dcc.Tab(label='QUARTERLY REVENUE FROM 2018-2021', value='tab-2', style=tabsOff_style, selected_style=tabsOn_style),
        dcc.Tab(label='NUMBER OF SUBSCRIBERS PER QUARTER FROM 2018-2021', value='tab-3', style=tabsOff_style, selected_style=tabsOn_style),
        ])
    ]),
    html.Div(id='effects-pandemicID'),
    
    html.Br(),
    html.Br(),
    
#NETFLIX IN THE UK
    html.H3(children="NETFLIX IN THE UK",id="Net", style={'textAlign':'left','color':colors['text']}),
    html.Br(),
    html.P(children="""Focusing on the UK alone, we look at the behavior of the Netflix UK subscribers as shown below. 
    Looking at the watch time and movie/TV shows the preference of each subscriber's shows and understanding 
    these trends would help us understand the content that individuals in the UK prefer. Ultimately helping producers, writers
    and directors, align the content they are creating to the preferences of the UK audidence.""", style=p_style),
    html.Br(),
    html.Div(className="Netflix-UKCN", children=[
        dcc.Tabs(id="netflix-tabs", value='tab-1', children=[
        dcc.Tab(label='TRENDING SHOWS BASED ON WATCH TIME', value='tab-1', style=tabsOff_style, selected_style=tabsOn_style),
        ])
    ]),
    html.Div(id='netflix-UKID'),  
],style=content_style)

#APP LAYOUT
app.layout = html.Div(className='container', children=[
    html.Br(),
    html.Br(),
    html.Br(),
    html.Img(src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAs8AAABACAYAAADlC5qTAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAAE/SURBVHhe7cixDYBAAMSw33/phxZdCgZwJDc5bxcAAPglJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAAsHICAAArJwAA8HHuA02nFr5ZXHseAAAAAElFTkSuQmCC", style=header2_img),
    html.Img(src="...", height=75, style=header_img),  
    content,
    sidebar,
])

#STREAMING SERVICES TREND
@app.callback(
    Output('streaming-servicesID', 'children'),
    Input('streaming-tabs', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.Br(),
            html.Div(children=
                    html.P(children="""The different streaming services, Netflix, Hulu, Prime Video, and Disney+, 
                    have a steady increase of shows over the years. The acquisition and production of different
                    movies and tv shows from these streaming sites have been more aggresive, this doesn't
                    mean that all of their shows have been of top quality. Some may say this is quantity 
                    over quality.""", style=p_style)),
            dcc.Graph(
                id='StreamingSites-release',
                figure = px.line(all_movies_year_df,"Year","counts",color="Site")
            ),
             html.Div(children=
                    html.P(children="""Among these services, the shows produced within the year 1920 and 1980 may be found in all streaming sites, 
                    but a very few number of them were to be found. However, the trend showed a number of movies produced within these years were mostly found in Prime. 
                    In the later years of movie production, the trend increased the number of shows among the 
                    streaming sites, and peaked at the highest number of shows found in all streaming sites around 2020.""", style=p_style))
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.Br(),
            html.Div(className="row", children=[
                html.P(children="""Based from the ratings of each streaming site, showed the comparison on the quality of shows produced. 
                Moreover, these ratings were taken from IMDb and Rotten Tomatoes.""", style=p_style),
                html.Br(),
                html.Div(className="col-4", children=[
                    dcc.RadioItems(
                        id="choice-radio1",
                        options=[
                            {'label': 'IMDb ', 'value': 0},
                            {'label': 'Rotten Tomatoes', 'value': 1},
                        ],
                        value=0,
                        labelStyle=radio_style 
                    ), 
                ])
            ]),
            html.Div(className="row", children=[
                html.Div(className="col-12", children=[
                    dcc.Graph(
                        id="average-score"
                    )
                ]),
                html.Br(),
                html.P(children="""Among the scores garnered in each streaming sites from IMDb, Disney+ has the highest average score compared to the other sites.
                Although the averages scores were almost tied, 
                this means that Disney+ contains better quality shows based from the ratings in IMDb.""", style=p_style), 
                html.Br(),
                html.P(children="""The ratings from Rotten Tomatoes were different from IMDb. Also, the ratings of 
                each of the streaming site were almost tied; however, Hulu has attained the highest rating, 
                which made this streaming site have better quality shows released compared with other streaming sites. """, style=p_style)
            ])
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.Br(),
            dcc.Dropdown(
                id='genre_total_dropdown',
                options=[
                    {'label': 'Biography', 'value': 0 },
                    {'label': 'Crime', 'value': 1},
                    {'label': 'Drama', 'value': 2 },
                    {'label': 'Action', 'value': 3 },
                    {'label': 'Sport', 'value': 4 },
                    {'label': 'Documentary', 'value': 5 },
                    {'label': 'Musical', 'value': 6},
                    {'label': 'History', 'value': 7 },
                    {'label': 'Romance', 'value': 8 },
                    {'label': 'Comedy', 'value': 9 },
                    {'label': 'Adventure', 'value': 10 },
                    {'label': 'Mystery', 'value': 11 },
                    {'label': 'Western', 'value': 12 },
                    {'label': 'Thriller', 'value': 13 },
                    {'label': 'Music', 'value': 14 },
                    {'label': 'Animation', 'value': 15 },
                    {'label': 'Fantasy', 'value': 16 },
                    {'label': 'Horror', 'value': 17 },
                    {'label': 'Family', 'value': 18 },
                    {'label': 'Short', 'value': 19 },
                    {'label': 'Talk', 'value': 20 },
                    {'label': 'Reality', 'value': 21 },
                    {'label': 'News', 'value': 22 },
                    {'label': 'Game', 'value': 23 },
                    {'label': 'Film', 'value': 24 },
                    {'label': 'Noir', 'value': 25 },
                ],
                value=0
            ),
            html.Br(),
            dcc.RadioItems(
                id = "Top_genre_radio",
                options=[
                    {'label': 'IMDb', 'value': 0},
                    {'label': 'Rotten Tomatoes', 'value': 1}
            ],
            value=0,
            labelStyle=radio_style
            ),
            dcc.Graph(
                id = "Genre_Sum", figure=genre_array_fig,
            ),
            dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in Genre_Top_Movies_df.loc[:,['Title', 'IMDb', 'Rotten Tomatoes']]],
                style_cell={'textAlign': 'left'},
                style_cell_conditional=[
                    {
                        'if': {'column_id': 'Title'},
                        'textAlign': 'left'
                    }
                ]
            ),
        ])

#EFFECTS OF THE PANDEMICs
@app.callback(
    Output('effects-pandemicID', 'children'),
    Input('effects-tabs', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.Div(className="row", children=[
                html.Div(className="col-4", children=[
                    html.Br(), 
                    dcc.RadioItems(
                        id="choice-radio2",
                        options=[
                            {'label': 'TV Shows & Movies ', 'value': 0},
                            {'label': 'Movies ', 'value': 1},
                            {'label': 'TV Shows ', 'value': 2}
                        ],
                        value=0,
                        labelStyle=radio_style
                    )
                ])
            ]),
            html.Div(className="row", children=[
                html.Div(className="col-12", children=[
                    html.Br(),
                    html.Br(),
                    html.P("""The graph shows the number of added shows per year on Netflix. 
                    The number of shows for the year 2021 is very few since the year still hasn't ended. 
                    The year 2019 has the highest record for the number of TV shows and movies added. 
                    It decreases during 2020 since there's a decrease in the number of movies added on that year.  
                    One of the reasons for this is the pandemic. Many movies were delayed since shootings were 
                    canceled to decrease the spreading of the virus.""", style=p_style),
                    dcc.Graph(
                        id="added-shows"
                    )
                ])
            ])
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.Br(),
            dcc.Dropdown(
                id="revenue-region-dropdown",
                options=[
                    {'label': 'All', 'value': 0},
                    {'label': 'United States and Canada', 'value': 1},
                    {'label': 'Europe, Middle East, and Africa', 'value': 2},
                    {'label': 'Latin America', 'value': 3},
                    {'label': 'Asia-Pacific', 'value': 4}
                ],
                value=0,
            ),
            html.Br(),
            dcc.RadioItems(
                id="revenue-change-radio",
                options=[
                    {'label': 'Revenue', 'value': 0},
                    {'label': 'Percent change in revenue', 'value': 1},
                ],
                value=0,
                labelStyle=radio_style
            ),
            html.Br(),
            html.Br(),
            html.P("""Overall, Netflix's revenue continue to increase over the past year. 
            A sudden increase in its revenue was recorded during the 1st Quarter of 2020. 
            About 5.64% increase was recorded during this quarter compared to the last quarter. 
            This is equivalent to 304.381M increase in revenue. The revenue increase during this 
            year since there's a  sudden increase in its subscriber during this quarter.""", style=p_style),
            dcc.Graph(
                    id="revenue"
            ),
            html.Br(),
            html.P("""Normally, with an increase of about 15.766M in subscribers, it is expected that the 
            revenue would hugely increase. However, the majority of the new subscribers came from low-revenue 
            regions such as Asia-Pacific. This resulted in a much lower increase in the revenue than 
            expected during the 1st quarter of 2020.""", style=p_style),
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.Br(),
            dcc.Dropdown(
                id="subscriber-region-dropdown",
                options=[
                    {'label': 'All', 'value': 0},
                    {'label': 'United States and Canada', 'value': 1},
                    {'label': 'Europe, Middle East, and Africa', 'value': 2},
                    {'label': 'Latin America', 'value': 3},
                    {'label': 'Asia-Pacific', 'value': 4}
                ],
                value=0,
            ),
            html.Br(),
            dcc.RadioItems(
                id="subscriber-change-radio",
                options=[
                    {'label': 'Total number of subscribers', 'value': 0},
                    {'label': 'Percent change in number of subscribers', 'value': 1},
                ],
                value=0,
                labelStyle=radio_style
            ),
            html.Br(),
            html.Br(),
            html.P("""The total number of subscribers for all regions increases over time. 
            Looking at the graph a sudden increase in the total number of subscribers happened during the 
            1st quarter of 2020. About 9.46% increase in the total number of subscribers added on that quarter 
            which is equivalent to a total of 15.766M new subscribers. During this time, the pandemic started 
            and many countries imposed lockdown. Many people subscribe during this time to continue watching 
            their favorite TV shows and movies.""", style=p_style),
            html.P("""The Asia-Pacific recorded the highest increase in the number of subscribers during the 
            1st quarter of 2020, having a 22.19% increase compared to the last quarter. It is followed by Europe, 
            Middle East, and Africa with a 13.43% increase. Next is Latin America with 9.23%. Lastly, the 
            United States and Canada with 3.41%. """, style=p_style),
            dcc.Graph(
                    id="subscribers"
            ),
            html.Br(),
            html.P("""Even though the number of United States and Canadian Netflix subscribers increases over time, 
            the percent change in the number of subscribers during the 4th quarter of 2019 decreases compared to 
            the last quarter. Only 548M  subscribers were added this quarter which is lower than their projected 
            subscriber growth. This is due to increased competition. Disney+ was released in the latter part of 
            the 4th quarter of 2019. """, style=p_style),
            html.P("""Because of the sudden increase in the number of subscribers during the lockdown on 
            the 1st quarter of 2020.  Netflix experienced a lumpiness in its subscriber's growth 
            especially during the 1st half of 2021.""", style=p_style),
        ])

#NETFLIX IN THE UK
@app.callback(
    Output('netflix-UKID', 'children'),
    Input('netflix-tabs', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.Button('Top 10', id='Top_10_Button', n_clicks=0),
            html.Button('Bottom 10', id='Bottom_10_Button', n_clicks=0),
            html.Br(),
            html.Br(),
            html.P(id="p1", style=p_style),
            dcc.Graph(
                id = 'Top_Bottom',
            ),
            html.P(id="p2", style=p_style),
        ])
    
#average score of each streaming service
@app.callback(
    # id, property
    Output('average-score', 'figure'),
    Input('choice-radio1', 'value')
)
def update_averagescore_graph(choice):
    if choice == 0:
        fig = IMDb_Graph2

    elif choice == 1:
        fig = Rottentomatoes2

    fig.update_layout(transition_duration=1000)
    return fig
    
#number of shows released by year
@app.callback(
    # id, property
    Output('added-shows', 'figure'),
    Input('choice-radio2', 'value')
)
def update_netflix_graph(choice):
    if choice == 0:
        # Create Top 10 Fast Food Restaurants in the USA
        netflix = netflix_show.groupby('year_added')['count'].agg('sum').reset_index(name='count')
        netflix.columns = ["Year", "Number of Tv Shows and Movies added"]
        fig = px.bar(netflix, x="Year", y="Number of Tv Shows and Movies added")

    elif choice == 1:
        netflix = netflix_show[netflix_show.type == "Movie"].reset_index(drop=True)
        netflix.drop('type', inplace=True, axis=1)
        netflix.columns = ["Year", "Number of Movies added"]

        fig = px.bar(netflix, x="Year", y="Number of Movies added")

    else:
        netflix = netflix_show[netflix_show.type == "TV Show"].reset_index(drop=True)
        netflix.drop('type', inplace=True, axis=1)
        netflix.columns = ["Year", "Number of TV Shows added"]

        fig = px.bar(netflix, x="Year", y="Number of TV Shows added")
    fig.update_layout(transition_duration=1000)
    return fig    

#quarterly revenue from 2018-2021
@app.callback(
    Output("revenue", "figure"),
    [Input("revenue-region-dropdown", "value"),
    Input("revenue-change-radio", "value")
    ]
)
def update_revenue_graph(region,preference):
    if region == 0:
        revfilDF = subsrevDF.groupby('year-quarter').sum().reset_index()
        for i in range(1, len(revfilDF)):
            a = revfilDF.loc[i - 1, "revenue"]
            b = revfilDF.loc[i, "revenue"]
            revfilDF.loc[i, "percent change in revenue"] = (b - a) / a * 100
        if preference == 0:
            fig = px.bar(revfilDF, x="year-quarter", y="revenue")
        else:
            fig = px.bar(revfilDF, x="year-quarter", y="percent change in revenue")

    if region == 1:
        revfilDF = subsrevDF[subsrevDF["region"]=="United States and Canada"].reset_index()
        if preference == 0:
            fig = px.bar(revfilDF, x="year-quarter", y="revenue")
        else:
            fig = px.bar(revfilDF, x="year-quarter", y="percent change in revenue")

    if region == 2:
        revfilDF = subsrevDF[subsrevDF["region"]=="Europe, Middle East, and Africa"].reset_index()
        if preference == 0:
            fig = px.bar(revfilDF, x="year-quarter", y="revenue")
        else:
            fig = px.bar(revfilDF, x="year-quarter", y="percent change in revenue")

    if region == 3:
        revfilDF = subsrevDF[subsrevDF["region"]=="Latin America"].reset_index()
        if preference == 0:
            fig = px.bar(revfilDF, x="year-quarter", y="revenue")
        else:
            fig = px.bar(revfilDF, x="year-quarter", y="percent change in revenue")

    if region == 4:
        revfilDF = subsrevDF[subsrevDF["region"]=="Asia-Pacific"].reset_index()
        if preference == 0:
            fig = px.bar(revfilDF, x="year-quarter", y="revenue")
        else:
            fig = px.bar(revfilDF, x="year-quarter", y="percent change in revenue")

    return fig

#trending shows based on watch time
@app.callback(
    Output("table", "data"),
    [Input("genre_total_dropdown", "value"),
    Input("Top_genre_radio", "value")
    ]
)
def Update_genre_table(genre_filter,scoring_filter):
    if(genre_filter == 0):
        Genre_Top_Movies_df = total_genre[(total_genre["Biography"]>0)]
    elif(genre_filter == 1):
        Genre_Top_Movies_df = total_genre[(total_genre["Crime"]>0)]
    elif(genre_filter == 2):
        Genre_Top_Movies_df = total_genre[(total_genre["Drama"]>0)]
    elif(genre_filter == 3):
        Genre_Top_Movies_df = total_genre[(total_genre["Action"]>0)]
    elif(genre_filter == 4):
        Genre_Top_Movies_df = total_genre[(total_genre["Sport"]>0)]
    elif(genre_filter == 5):
        Genre_Top_Movies_df = total_genre[(total_genre["Documentary"]>0)]
    elif(genre_filter == 6):
        Genre_Top_Movies_df = total_genre[(total_genre["Musical"]>0)]
    elif(genre_filter == 7):
        Genre_Top_Movies_df = total_genre[(total_genre["History"]>0)]
    elif(genre_filter == 8):
        Genre_Top_Movies_df = total_genre[(total_genre["Romance"]>0)]
    elif(genre_filter == 9):
        Genre_Top_Movies_df = total_genre[(total_genre["Comedy"]>0)]
    elif(genre_filter == 10):
        Genre_Top_Movies_df = total_genre[(total_genre["Adventure"]>0)]
    elif(genre_filter == 11):
        Genre_Top_Movies_df = total_genre[(total_genre["Mystery"]>0)]
    elif(genre_filter == 12):
        Genre_Top_Movies_df = total_genre[(total_genre["Western"]>0)]
    elif(genre_filter == 13):
        Genre_Top_Movies_df = total_genre[(total_genre["Thriller"]>0)]
    elif(genre_filter == 14):
        Genre_Top_Movies_df = total_genre[(total_genre["Music"]>0)]
    elif(genre_filter == 15):
        Genre_Top_Movies_df = total_genre[(total_genre["Animation"]>0)]
    elif(genre_filter == 16):
        Genre_Top_Movies_df = total_genre[(total_genre["Fantasy"]>0)]
    elif(genre_filter == 17):
        Genre_Top_Movies_df = total_genre[(total_genre["Horror"]>0)]
    elif(genre_filter == 18):
        Genre_Top_Movies_df = total_genre[(total_genre["Family"]>0)]
    elif(genre_filter == 19):
        Genre_Top_Movies_df = total_genre[(total_genre["Short"]>0)]
    elif(genre_filter == 20):
        Genre_Top_Movies_df = total_genre[(total_genre["Talk"]>0)]
    elif(genre_filter == 21):
        Genre_Top_Movies_df = total_genre[(total_genre["Reality"]>0)]
    elif(genre_filter == 22):
        Genre_Top_Movies_df = total_genre[(total_genre["News"]>0)]
    elif(genre_filter == 23):
        Genre_Top_Movies_df = total_genre[(total_genre["Game"]>0)]
    elif(genre_filter == 24):
        Genre_Top_Movies_df = total_genre[(total_genre["Film"]>0)]
    elif(genre_filter == 25):
        Genre_Top_Movies_df = total_genre[(total_genre["Noir"]>0)]
    else:
        Genre_Top_Movies_df = total_genre[(total_genre["Drama"]>0)]

    if (scoring_filter == 0):
        Genre_Top_Movies_df = Genre_Top_Movies_df.sort_values('IMDb', ascending = False)
    elif (scoring_filter == 1):
        Genre_Top_Movies_df = Genre_Top_Movies_df.sort_values('Rotten Tomatoes', ascending = False)
    if (Genre_Top_Movies_df["Title"].count() > 9):
            Genre_Top_Movies_df = Genre_Top_Movies_df.head(10)
    else:
        Genre_Top_Movies_df = Genre_Top_Movies_df.head(Genre_Top_Movies_df["Title"].count())
    return Genre_Top_Movies_df.to_dict('records')

#number of suscribers per quarter
@app.callback(
    Output("subscribers", "figure"),
    [Input("subscriber-region-dropdown", "value"),
    Input("subscriber-change-radio", "value")
    ]
)
def update_subscriber_graph(region,preference):
    if region == 0:
        subfilDF = subsrevDF.groupby('year-quarter').sum().reset_index()
        for i in range(1, len(subfilDF)):
            a = subfilDF.loc[i - 1, "number of subscribers"]
            b = subfilDF.loc[i, "number of subscribers"]
            subfilDF.loc[i, "percent change in subscribers"] = (b - a) / a * 100
        if preference == 0:
            fig = px.bar(subfilDF, x="year-quarter", y="number of subscribers")
        else:
            fig = px.bar(subfilDF, x="year-quarter", y="percent change in subscribers")

    if region == 1:
        subfilDF = subsrevDF[subsrevDF["region"]=="United States and Canada"].reset_index()
        if preference == 0:
            fig = px.bar(subfilDF, x="year-quarter", y="number of subscribers")
        else:
            fig = px.bar(subfilDF, x="year-quarter", y="percent change in subscribers")

    if region == 2:
        subfilDF = subsrevDF[subsrevDF["region"]=="Europe, Middle East, and Africa"].reset_index()
        if preference == 0:
            fig = px.bar(subfilDF, x="year-quarter", y="number of subscribers")
        else:
            fig = px.bar(subfilDF, x="year-quarter", y="percent change in subscribers")

    if region == 3:
        subfilDF = subsrevDF[subsrevDF["region"]=="Latin America"].reset_index()
        if preference == 0:
            fig = px.bar(subfilDF, x="year-quarter", y="number of subscribers")
        else:
            fig = px.bar(subfilDF, x="year-quarter", y="percent change in subscribers")

    if region == 4:
        subfilDF = subsrevDF[subsrevDF["region"]=="Asia-Pacific"].reset_index()
        if preference == 0:
            fig = px.bar(subfilDF, x="year-quarter", y="number of subscribers")
        else:
            fig = px.bar(subfilDF, x="year-quarter", y="percent change in subscribers")

    return fig

#trending shows based on watch time
@app.callback(
    Output('Top_Bottom', 'figure'),
    Input('Top_10_Button', 'n_clicks'),
    Input('Bottom_10_Button', 'n_clicks')
)
def displayClick(btn1, btn2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'Top_10_Button' in changed_id:
        #Top_fig.columns = ["Duration", "Title of Show"]
        Top_fig = px.bar(Top_10, x="duration",y="title")
        return Top_fig
    elif 'Bottom_10_Button' in changed_id:
        #Bottom_fig.columns = ["Duration", "Title of Show"]
        Bottom_fig = px.bar(Bot_10, x="duration",y="title")
        return Bottom_fig
    else:
        #Top_fig.columns = ["Duration", "Title of Show"]
        Top_fig = px.bar(Top_10, x="duration",y="title")
        return Top_fig
@app.callback(
    Output('p1', 'children'),
    Input('Top_10_Button', 'n_clicks'),
    Input('Bottom_10_Button', 'n_clicks')
)
def displayClick(btn1, btn2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'Top_10_Button' in changed_id:
        return """The horizontal bar graph above shows the top 10 movies and series in the UK in terms of total 
        watch time, and as you can see, the top 1 is “Black Mirror: Bandersnatch”, followed by “Hot Fuzz”, 
        “Avengers: Age of Ultron”, “Annihilation”, “Deadpool”, “Bird Box”, “The Hitman’s Bodyguard”, 
        “13 Reasons Why: Beyond the Reasons”, “FYRE: The Greatest Party That Never Happened”, and “Bright”."""
    elif 'Bottom_10_Button' in changed_id:
        return """The horizontal bar graph above shows the bottom 10 movies and series in the UK in terms of total 
        watch time, and as you can see, the bottom 1 is tied with “Simon Birch”, “Hyena Road”, 
        “Meet the Fockers”, “The Bund”, “A la mala”, and “Outlander”, with bottom second is also tied with 
        “TinkerBell and the Great Fairy Rescue”, “Great Human Odyssey”, “Parrot Heads”, and 
        “El club de los buenos inflieles”. """
    else:
        return """The horizontal bar graph above shows the top 10 movies and series in the UK in terms of total 
        watch time, and as you can see, the top 1 is “Black Mirror: Bandersnatch”, followed by “Hot Fuzz”, 
        “Avengers: Age of Ultron”, “Annihilation”, “Deadpool”, “Bird Box”, “The Hitman’s Bodyguard”, 
        “13 Reasons Why: Beyond the Reasons”, “FYRE: The Greatest Party That Never Happened”, and “Bright”."""
@app.callback(
    Output('p2', 'children'),
    Input('Top_10_Button', 'n_clicks'),
    Input('Bottom_10_Button', 'n_clicks')
)
def displayClick(btn1, btn2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'Top_10_Button' in changed_id:
        return """This could be because the ratings of each media in IMDb range from 6 to 8 out of 10, 
        particularly with Balck mirror with a IMDb score of 7.1/10. Another reason as to why these could be the 
        most watched content could be because of the replay value, providing the audience with eye-catching and 
        appealing story, setting, or characters. Another could be because of clever marketing strategies that catch 
        the eyes of consumers. Something to note, Majority of the media in the top 10 movies and series have genres 
        like action, adventure, sci-fi, and drama, which could be an indication of the trends consumers in the UK enjoy."""
    elif 'Bottom_10_Button' in changed_id:
        return """It is difficult to tell why these shows are at the bottom 10 in terms of watch time. 
        One possibility could be the date of release of the movies, some going as far back as 1971, 
        hence not a lot of people know the movie. Although, some of the shows in the list have a high 
        rating in both IMDb and Rotten tomatoes, one prime example being “The Last Picture Show” with 
        a score of 8/10 and 100% from IMDb and Rotten Tomatoes, respectively. One reason as to why they 
        are in the bottom 10 could be because of the relevance of the film, how they are not timely in today's setting."""
    else:
        return """TThis could be because the ratings of each media in IMDb range from 6 to 8 out of 10, 
        particularly with Balck mirror with a IMDb score of 7.1/10. Another reason as to why these could be the 
        most watched content could be because of the replay value, providing the audience with eye-catching and 
        appealing story, setting, or characters. Another could be because of clever marketing strategies that catch 
        the eyes of consumers. Something to note, Majority of the media in the top 10 movies and series have genres 
        like action, adventure, sci-fi, and drama, which could be an indication of the trends consumers in the UK enjoy."""

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Sep/2021 23:11:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:59] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:59] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:11:59] "POST /_dash-update-component HTTP/1.1"